In [5]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

import pandas as pd

# Load the annotations file
annotations_file_path = 'Osteo_Fibro_MSC_Annotations.txt'
annotations_df = pd.read_csv(annotations_file_path, delimiter='\t')

In [ ]:
# Iterate through each row in the annotations file
loaded_images = {}

for index, row in annotations_df.iterrows():
    print(index)
    # Extract necessary information from the row
    image_path = f"Bone_Annotations/{row['Image'].split(' ')[0]} - resolution #1-bone-labels.png"
    
    # Load the image only if it hasn't been loaded before
    if image_path not in loaded_images:
        loaded_images[image_path] = Image.open(image_path)

    x_coordinate = int(row['centroid_x_pixel'])
    y_coordinate = int(row['centroid_y_pixel'])

    # Get the pixel value at the specified coordinates
    pixel_value = loaded_images[image_path].getpixel((x_coordinate, y_coordinate))

    # Update the DataFrame with the pixel value
    annotations_df.at[index, 'Pixel_Value'] = pixel_value

In [7]:
# Write the updated DataFrame to a new text file
output_file_path = 'Osteo_Fibro_MSC_Annotations_2.txt'
annotations_df.to_csv(output_file_path, sep='\t', index=False)

In [8]:
#Reload and print out the points that are overlapping bone 
# Load the updated annotations file
updated_annotations_file_path = 'Osteo_Fibro_MSC_Annotations_2.txt'
updated_annotations_df = pd.read_csv(updated_annotations_file_path, delimiter='\t')

# Filter rows where Pixel_Value equals 1.0
filtered_df = updated_annotations_df[updated_annotations_df['Pixel_Value'] == 1.0]

# Create a new DataFrame with required columns x0, y0, x1, y1
new_df = pd.DataFrame({
    'sample' : filtered_df['Sample'],
    'celltype' : filtered_df['Classification'],
    'x0': filtered_df['centroid_x_pixel'],
    'y0': filtered_df['centroid_y_pixel'],
    'x1': filtered_df['centroid_x_pixel'],
    'y1': filtered_df['centroid_y_pixel'] + 1
})

# Write the new DataFrame to a new text file
output_file_path = 'Osteo_Fibro_MSC_on_Bone.txt'
new_df.to_csv(output_file_path, sep='\t', index=False)